In [2]:
import pyyolo
import numpy as np
import sys
import cv2
from scipy.ndimage.measurements import label
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import collections

q= collections.deque(maxlen=10)

In [3]:
darknet_path = './darknet'
datacfg = 'cfg/coco.data'
cfgfile = 'cfg/tiny-yolo.cfg'
weightfile = '../tiny-yolo.weights'
thresh = 0.24
hier_thresh = 0.5

pyyolo.init(darknet_path, datacfg, cfgfile, weightfile)


In [4]:
def yolo_predict(image):
    cv2.imwrite('temp.jpg',image)
    outputs = pyyolo.test('temp.jpg', thresh, hier_thresh, 0)
    bbox=[]
    for output in outputs:
        if(output['class']=='car'):
            bbox.append(((output['left'], output['top']),(output['right'], output['bottom'])))
        #print(output)
        #print('left',output['left'],'top',output['top'])
        #print('right',output['right'],'bottom',output['bottom'])
        #cv2.rectangle(img, (output['left'], output['top']), (output['right'], output['bottom']), (255,0,0), 2)
        #cv2.imwrite("./op/"+filename,img)
    return bbox

In [5]:
# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img


In [9]:
import time
def pipeline1(image):
    draw_img = np.copy(image)
    
    res=[]
    
    hot_windows = yolo_predict(image)
    
                    
    #print(hot_windows)
#     if(len(hot_windows)>0):
#         res.append(hot_windows)
        #print(hot_windows)
        #print(res.shape)

    #print(res)
#     ans=[]
#     ans.append(res[0])
#     for i in res[1:]:
#         ans[0]+=i
    #print(ans[0])

    #window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    
    window_img = draw_boxes(draw_img, hot_windows, color=(0, 0, 255), thick=6)  
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    heat = add_heat(heat,hot_windows)
    
    q.append(heat)
    if(len(q)==10):
        
        heat=np.sum(q,axis=0)
       

        # Apply threshold to help remove false positives
        heat = apply_threshold(heat,8)

        # Visualize the heatmap when displaying    
        heatmap = np.clip(heat, 0, 255)

        # Find final boxes from heatmap using label function
        labels = label(heatmap)
        draw_img = draw_labeled_bboxes(np.copy(image), labels)
    #cv2.imwrite('./tes/'+str(time.time())+'.jpg',window_img)
    
    return window_img
  

In [7]:
#!sudo pip install moviepy

In [10]:
# Import everything needed to edit/save/watch video clips
import imageio 
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML


Imageio: 'ffmpeg.linux64' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg.linux64 (27.2 MB)
Downloading: 8192/28549024 bytes (0.016384/28549024 bytes (0.1%32768/28549024 bytes (0.1%65536/28549024 bytes (0.2%98304/28549024 bytes (0.3%131072/28549024 bytes (0.5163840/28549024 bytes (0.6196608/28549024 bytes (0.7229376/28549024 bytes (0.8245760/28549024 bytes (0.9294912/28549024 bytes (1.0327680/28549024 bytes (1.1393216/28549024 bytes (1.4409600/28549024 bytes (1.4442368/28549024 bytes (1.5507904/28549024 bytes (1.8606208/28549024 bytes (2.1720896/28549024 bytes (2.5819200/28549024 bytes (2.9950272/28549024 bytes (3.31081344/28549024 bytes (3.8%1212416/28549024 bytes (4.2%1343488/28549024 bytes (4.7%1474560/28549024 bytes (5.2%1622016/28549024 bytes (5.7%1785856/28549024 bytes (6.3%1933312/28549024 bytes (6.8%2080768/28549024 bytes (7.3%2228224/28549024 bytes (7.8%2375680/28549024 bytes (8.3%252

In [11]:
ind=0
white_output = 'yolo_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(pipeline1) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video yolo_output.mp4
[MoviePy] Writing video yolo_output.mp4


100%|█████████▉| 1260/1261 [1:54:00<00:06,  6.28s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yolo_output.mp4 

CPU times: user 1h 51min 49s, sys: 1min 39s, total: 1h 53min 29s
Wall time: 1h 54min 3s
